# Spaceship Lightgbm Submission

In [ ]:
!pip install lightgbm==4.3.0


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix
from lightgbm import LGBMClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import MinMaxScaler

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import optuna

# Load datasets
X_df = pd.read_csv('spaceship_train_X_v2.csv')
y_df = pd.read_csv('spaceship_train_y.csv')
X_submission = pd.read_csv('spaceship_test_X_v2.csv')

# Drop unnecessary columns
y_df.drop('Unnamed: 0', axis=1, inplace=True)
X_df.drop('PassengerId', axis=1, inplace=True)
X_submission_ids = X_submission['PassengerId']
X_submission.drop('PassengerId', axis=1, inplace=True)

# Split data
X_train, X_val, y_train, y_val = train_test_split(X_df, y_df['Transported'], test_size=0.2, random_state=42)

# Standardize data
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_submission_scaled = scaler.transform(X_submission)

# Define objective function for Optuna
def objective(trial):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'max_depth': trial.suggest_int('max_depth', 3, 20),
        'num_leaves': trial.suggest_int('num_leaves', 20, 300),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 1.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 1.0)
    }

    lgbm = LGBMClassifier(**param, random_state=42)
    lgbm.fit(X_train_scaled, y_train)
    y_pred = lgbm.predict(X_val_scaled)
    accuracy = accuracy_score(y_val, y_pred)
    return accuracy

# Optimize hyperparameters with Optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=200)

# Print best hyperparameters
print('Best hyperparameters:', study.best_params)

# Train final model with best hyperparameters
best_params = study.best_params
lgbm_optimized = LGBMClassifier(**best_params, random_state=42)
lgbm_optimized.fit(X_train_scaled, y_train)

# Evaluate the model
y_pred_optimized = lgbm_optimized.predict(X_val_scaled)
print("Accuracy on validation set:", accuracy_score(y_val, y_pred_optimized))
print("Classification Report:\n", classification_report(y_val, y_pred_optimized))
print("Confusion Matrix:\n", confusion_matrix(y_val, y_pred_optimized))

# Fit the model on all training data
X_df_scaled = scaler.fit_transform(X_df)
lgbm_optimized.fit(X_df_scaled, y_df['Transported'])

# Predict on submission data
y_submission_pred = lgbm_optimized.predict(X_submission_scaled)

# Create submission file
submission_dict = {'PassengerId': X_submission_ids, 'Transported': y_submission_pred.astype('bool')}
submission_df = pd.DataFrame(submission_dict)
submission_df.to_csv('spaceship_lgbm_optimized.csv', index=False)


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

In [ ]:
!pip install catboost==1.2.5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 14.4 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [ ]:
!pip install optuna==3.6.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 37.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 738.9/738.9 kB 94.6 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import optuna

# Load datasets
X_df = pd.read_csv('spaceship_train_X_v2.csv')
y_df = pd.read_csv('spaceship_train_y.csv')
X_submission = pd.read_csv('spaceship_test_X_v2.csv')

# Drop unnecessary columns
y_df.drop('Unnamed: 0', axis=1, inplace=True)
X_df.drop('PassengerId', axis=1, inplace=True)
X_submission_ids = X_submission['PassengerId']
X_submission.drop('PassengerId', axis=1, inplace=True)

# Split data
X_train, X_val, y_train, y_val = train_test_split(X_df, y_df['Transported'], test_size=0.2, random_state=42)

# Standardize data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_submission_scaled = scaler.transform(X_submission)

# Define objective function for Optuna
def objective(trial):
    param = {
        'iterations': trial.suggest_int('iterations', 100, 1000),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'depth': trial.suggest_int('depth', 3, 16),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-5, 100),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.0, 1.0),
        'random_strength': trial.suggest_float('random_strength', 0.0, 1.0),
        'border_count': trial.suggest_int('border_count', 1, 255)
    }

    catboost = CatBoostClassifier(**param, random_state=42, verbose=0)
    catboost.fit(X_train_scaled, y_train)
    y_pred = catboost.predict(X_val_scaled)
    accuracy = accuracy_score(y_val, y_pred)
    return accuracy

# Optimize hyperparameters with Optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=10)

# Print best hyperparameters
print('Best hyperparameters:', study.best_params)

# Train final model with best hyperparameters
best_params = study.best_params
catboost_optimized = CatBoostClassifier(**best_params, random_state=42, verbose=0)
catboost_optimized.fit(X_train_scaled, y_train)

# Evaluate the model
y_pred_optimized = catboost_optimized.predict(X_val_scaled)
print("Accuracy on validation set:", accuracy_score(y_val, y_pred_optimized))
print("Classification Report:\n", classification_report(y_val, y_pred_optimized))
print("Confusion Matrix:\n", confusion_matrix(y_val, y_pred_optimized))

# Fit the model on all training data
X_df_scaled = scaler.fit_transform(X_df)
catboost_optimized.fit(X_df_scaled, y_df['Transported'])

# Predict on submission data
y_submission_pred = catboost_optimized.predict(X_submission_scaled)

# Create submission file
submission_dict = {'PassengerId': X_submission_ids, 'Transported': y_submission_pred.astype('bool')}
submission_df = pd.DataFrame(submission_dict)
submission_df.to_csv('spaceship_catboost_optimized.csv', index=False)


/shared-libs/python3.9/py/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2024-05-17 16:55:39,000] A new study created in memory with name: no-name-f5985eb2-1c72-4967-8c52-d0495bd659fd
[I 2024-05-17 16:57:28,554] Trial 0 finished with value: 0.7901092581943646 and parameters: {'iterations': 747, 'learning_rate': 0.19127648890743468, 'depth': 13, 'l2_leaf_reg': 31.53700682896767, 'bagging_temperature': 0.1921633056602322, 'random_strength': 0.9017837325341022, 'border_count': 51}. Best is trial 0 with value: 0.7901092581943646.
[I 2024-05-17 16:57:33,256] Trial 1 finished with value: 0.7987349051178838 and parameters: {'iterations': 811, 'learning_rate': 0.16120195322541006, 'depth': 4, 'l2_leaf_reg': 64.80933581315008, 'bagging_temperature': 0.5989308264972837, 'random_strength': 0.6045036910138146, '

In [ ]:
# Evaluate the model
print("Accuracy:", accuracy_score(y_val, y_pred_optimized))
print("Classification Report:\n", classification_report(y_val, y_pred_optimized))
print("Confusion Matrix:\n", confusion_matrix(y_val, y_pred_optimized))

Accuracy: 0.8056354226566993
Classification Report:
               precision    recall  f1-score   support

           0       0.81      0.80      0.80       861
           1       0.81      0.81      0.81       878

    accuracy                           0.81      1739
   macro avg       0.81      0.81      0.81      1739
weighted avg       0.81      0.81      0.81      1739

Confusion Matrix:
 [[690 171]
 [167 711]]


# Bring the Data In

In [ ]:
X_df = pd.read_csv('spaceship_train_X_v2.csv')
y_df = pd.read_csv('spaceship_train_y.csv')
X_submission = pd.read_csv('spaceship_test_X_v2.csv')

y_df.drop('Unnamed: 0', axis=1, inplace=True)
X_df.head()

,PassengerId,CryoSleep,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Expenditure,NoSpending,...,CabinSide_Z,CabinDeck_A,CabinDeck_B,CabinDeck_C,CabinDeck_D,CabinDeck_E,CabinDeck_F,CabinDeck_G,CabinDeck_T,AgeEncoded
0,0001_01,0,0,-0.337530,-0.283865,-0.287383,-0.273826,-0.265831,-0.518357,1,...,0,0,1,0,0,0,0,0,0,3.0
1,0002_01,0,0,-0.172455,-0.278226,-0.245562,0.213965,-0.227033,-0.256582,0,...,0,0,0,0,0,0,1,0,0,2.0
2,0003_01,0,1,-0.272409,1.956643,-0.287383,5.692512,-0.222625,3.174596,0,...,0,1,0,0,0,0,0,0,0,4.0
3,0003_02,0,0,-0.337530,0.519986,0.333240,2.684020,-0.095651,1.322607,0,...,0,1,0,0,0,0,0,0,0,2.0
4,0004_01,0,0,0.121349,-0.240007,-0.034784,0.228181,-0.264067,-0.130318,0,...,0,0,0,0,0,0,1,0,0,1.0


In [ ]:
from sklearn.model_selection import train_test_split

# Memisahkan data menjadi train dan validation set
X_train, X_val, y_train, y_val = train_test_split(X_df, y_df, test_size=0.2, random_state=42)
# Mengubah target menjadi 1D array
y_train = y_train['Transported'].values
y_test = y_val['Transported'].values

# Model Training

In [ ]:

# Membuat pipeline

pipeline = Pipeline([
    ('scaler', MinMaxScaler()),
    ('classifier', LGBMClassifier(random_state=42))
])



param_grid = {
    'classifier__n_estimators': [500],
    'classifier__max_depth': [10],
    'classifier__learning_rate': [0.01],
    'classifier__subsample': [0.8],
    'classifier__colsample_bytree': [1]
}

# Menggunakan StratifiedKFold untuk handling imbalanced data
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)


In [ ]:
# Membuat GridSearchCV dengan cv yang lebih robust
grid_search = GridSearchCV(pipeline, param_grid, cv=cv, scoring='accuracy', verbose=1)
grid_search.fit(X_train, y_train)

# Menampilkan parameter terbaik dan skor
print("Best parameters:", grid_search.best_params_)
print("Best cross-validation score: {:.2f}".format(grid_search.best_score_))

Fitting 10 folds for each of 1 candidates, totalling 10 fits
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 3150, number of negative: 3108
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000606 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2126
[LightGBM] [Info] Number of data points in the train set: 6258, number of used features: 38
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.503356 -> initscore=0.013423
[LightGBM] [Info] Start training from score 0.013423
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=3

In [ ]:
# Menggunakan estimator terbaik untuk prediksi pada data test
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_val)

# Evaluasi model pada data test
print("Accuracy on test set:", accuracy_score(y_val, y_pred))
print("Classification Report:\n", classification_report(y_val, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_val, y_pred))

# Fit the model on all training data
grid_search.fit(X_df, y_df)

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
Accuracy on test set: 0.7993099482461185
Classification Report:
               precision    recall  f1-score   support

           0       0.81      0.78      0.79       861
           1       0.79      0.82      0.80       878

    accuracy                           0.80      1739
   macro avg       0.80      0.80      0.80      1739
weighted avg       0.80      0.80      0.80      1739

Confusion Matrix:
 [[671 190]
 [159 719]]
Fitting 10 folds for each of 1 candidates, totalling 10 fits
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 3940, number of negative: 3883
[LightGBM] [Info] Auto-choosing row-wise multi

GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=42, shuffle=True),
             estimator=Pipeline(steps=[('scaler', MinMaxScaler()),
                                       ('classifier',
                                        LGBMClassifier(random_state=42))]),
             param_grid={'classifier__colsample_bytree': [1],
                         'classifier__learning_rate': [0.01],
                         'classifier__max_depth': [10],
                         'classifier__n_estimators': [500],
                         'classifier__subsample': [0.8]},
             scoring='accuracy', verbose=1)

# Submission Prediction

In [ ]:
# For training, we use ALL data from spaceship_train_X_v2.csv and spaceship_train_y.csv
grid_search.fit(X_df, y_df['Transported'].values)
# Prediksi data submission
y_prediction = grid_search.predict(X_submission)
print(y_prediction)


Fitting 10 folds for each of 1 candidates, totalling 10 fits
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 3940, number of negative: 3883
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000703 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2126
[LightGBM] [Info] Number of data points in the train set: 7823, number of used features: 38
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.503643 -> initscore=0.014573
[LightGBM] [Info] Start training from score 0.014573
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=3

In [ ]:
submission_dict = {'PassengerId':X_submission['PassengerId'], 'Transported':y_prediction.astype('bool')}
submission_dict = pd.DataFrame(submission_dict)
submission_dict

,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,True
...,...,...
4272,9266_02,True
4273,9269_01,False
4274,9271_01,True
4275,9273_01,True


## Export CSV

In [ ]:
submission_dict.to_csv('spaceship_lightgbm6.csv', index=False)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=2a79941c-6614-47fe-9427-0e9f23998893' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>